In [1]:
import pandas as pd

**Import Data**

In [2]:
path = "./sample_user.csv"
sample_user = pd.read_csv(path, sep="\t")

In [3]:
len(set(sample_user.user_id))

67095

**Get Friends**

In [4]:
friends = list(map(lambda x: x.split(', '), sample_user.friends))

In [5]:
import itertools
fr = set(itertools.chain.from_iterable(friends))

In [6]:
u = set(sample_user.user_id)

In [7]:
d = {}
for i, a in enumerate(u | fr):
    d[a] = i

**Reassign index to User**

In [8]:
sample_user['id'] = list(map(lambda x: d[x], sample_user.user_id))
sample_user['friend_id'] = list(map(lambda x: set(map(lambda y: d[y], x.split(', '))), sample_user.friends))
sample_user.drop(columns=['user_id', 'friends'], inplace=True)

In [9]:
sample_user.to_json("cleaned_user.json")

In [10]:
path = "sample_review.csv"
reviews = pd.read_csv(path, sep="\t")

In [11]:
dfs = []

In [12]:
from tqdm import tqdm

In [13]:
for review in tqdm(reviews):
    dfs.append(reviews[reviews['user_id'].isin(u)])

  0%|          | 8/883363 [00:01<41:36:02,  5.90it/s]


In [14]:
n = 0
for c in dfs:
    n += c.shape[0]
print(n)

7066904


In [15]:
sample_reviews = pd.concat(dfs, axis=0)

In [16]:
sample_reviews['id'] = list(map(lambda x: d[x], sample_reviews.user_id))

In [17]:
sample_reviews.drop(columns=['user_id'], inplace=True)

**Table due to normalization**

In [18]:
import json

with open("user_id.json", "w") as fp:
    json.dump(d , fp)

In [19]:
business = sample_reviews['business_id'].unique()

In [20]:
b = {}
for i, a in enumerate(business):
    b[a] = i

In [24]:
with open("business_id.json", "w") as fp:
    json.dump(b , fp)

,cool,funny,stars,useful,id,bus_id
0,0,0,4.0,0,51444,0
1,0,0,4.0,1,1107773,1
2,2,5,5.0,2,4680855,2
3,0,0,5.0,0,2409675,3
4,0,0,4.0,0,3735729,4
...,...,...,...,...,...,...
883358,1,0,3.0,5,787821,102866
883359,1,0,5.0,1,2006307,97709
883360,2,0,4.0,3,4098917,95289
883361,0,0,5.0,1,1549047,102724


In [27]:
sample_reviews['bus_id'] = list(map(lambda x: b[x], sample_reviews.business_id))
sample_reviews.drop(columns=['business_id', 'date'], inplace=True)

In [30]:
sample_reviews.drop(columns=['Unnamed: 0'], inplace=True)

In [34]:
sample_reviews.reset_index(inplace=True)

In [35]:
sample_reviews.to_json("sample_review.json")

**Relationship between user and reviewed business**

In [1]:
import pandas as pd

In [2]:
review = pd.read_json("sample_review.json")

In [25]:
c = review[['id', 'bus_id']].groupby('id', as_index=False).agg(lambda df: list(df))

In [27]:
c['bus_new'] = list(map(lambda row: ", ".join(list(set(map(str, row)))), c['bus_id']))

In [28]:
c.drop(columns='bus_id', inplace=True)

In [29]:
c.to_csv('user_review_bus.csv', sep='\t')

In [30]:
c[c.id == 2531369]

,id,bus_new
30772,2531369,10595


In [31]:
c[c.id == 3550278]

,id,bus_new
43031,3550278,10595
